## Import modules

In [10]:
import numpy as np
import lime
import tensorflow as tf
from lime.lime_text import LimeTextExplainer

from transformers import BertTokenizerFast, TFBertForSequenceClassification, TextClassificationPipeline

In [ ]:
# load finetuned models from hugging face
tokenizer = BertTokenizerFast.from_pretrained('Tolerblanc/klue-bert-finetuned', from_tf=True)
model = TFBertForSequenceClassification.from_pretrained('Tolerblanc/klue-bert-finetuned', output_attentions=True)

In [11]:
# load finetuned models from local
tokenizer = BertTokenizerFast.from_pretrained('curse_detection/klue-bert-base', from_tf=True)
model = TFBertForSequenceClassification.from_pretrained('curse_detection/klue-bert-base', output_attentions=True)

Some layers from the model checkpoint at curse_detection/klue-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at curse_detection/klue-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


### Masking Functions with LIME

In [87]:
class_names = ['clean','curse']

def predictor(texts):
    inputs = tokenizer(texts, return_tensors="tf", padding=True)
    outputs = model(inputs)
    probas = tf.nn.softmax(outputs.logits).numpy()
    return probas

explainer = LimeTextExplainer(class_names=class_names)

def lime_masking(lime_explainer, prob_method, texts):
	exp = lime_explainer.explain_instance(texts, prob_method)
	feature_scores = exp.as_list()
	masked = "*" * len(feature_scores[0][0])
	texts = texts.replace(feature_scores[0][0], masked)
	for feature, score in feature_scores:
		print(feature, score)
		if score > 0.37:
			masked = "*" * len(feature)
			texts = texts.replace(feature, masked)
	
	return (texts)

### Wrapper Function to get clean result 

In [67]:
from transformers import TextClassificationPipeline

klue_bert_classifier = TextClassificationPipeline(
    tokenizer=tokenizer, 
    model=model, 
    framework='tf',
)

def get_result(classifier, masking_method, explainer, prob_method, text):
	output = classifier(text)[0]
	if output['label'] == 'LABEL_1':
		text = masking_method(explainer, prob_method, text)
	return (text)

In [86]:
get_result(klue_bert_classifier, lime_masking, explainer, predictor, "내 앞에서 만두 먹네 개시키")

'내 앞에서 만두 ** ***'

In [90]:
print(get_result(klue_bert_classifier, lime_masking, explainer, predictor, "마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나"))
print()
print(get_result(klue_bert_classifier, lime_masking, explainer, predictor, "마 씨발련아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나"))

씨발련 0.3035191844508606
대하이햄하고 0.13552007294222623
해밨나 0.12440237129615614
니 0.09173280537749583
몇평이고 0.06997125798159522
맷개드갔노 -0.025232380037995952
마 0.025212365564952707
아 -0.019426309057547272
마 *** 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나

씨발련아 0.2610076683030981
대하이햄하고 0.1560522748491922
해밨나 0.12329999754743852
니 0.07214382352365727
몇평이고 0.055339407872269986
마 0.02281098961764866
맷개드갔노 0.010028274612021285
마 **** 몇평이고 맷개드갔노 니 대하이햄하고 해밨나


In [85]:
get_result(klue_bert_classifier, lime_masking, explainer, predictor, "나라 생긴지 몇 년이나 됐다고 저런 전통도 있었냐 ㅋㅋ ㅂㄹㅈ ㅋㅋㅋ")

'나라 생긴지 몇 년이나 됐다고 저런 전통도 있었냐 ㅋㅋ *** ㅋㅋㅋ'

In [77]:
get_result(klue_bert_classifier, lime_masking, explainer, predictor, \
	"내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 새끼는 월급 못받고 굶어 뒈져야 돼 병신새끼야.")

'내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 새끼는 월급 못받고 굶어 뒈져야 돼 *****.'

In [78]:
get_result(klue_bert_classifier, lime_masking, explainer, predictor, \
	"오마쥬가 뭔진 앎?")

'오마쥬가 뭔진 앎?'

In [83]:
get_result(klue_bert_classifier, lime_masking, explainer, predictor, \
	"지랄 났다 진짜 시발 대체 종강은 언제하냐?")

'지랄 났다 진짜 ** 대체 종강은 언제하냐?'

In [88]:
get_result(klue_bert_classifier, lime_masking, explainer, predictor, \
	"씨벌련이 닉변하고 아침에 올린거 토씨하나 안바꾸고  그대-로 좆중복 조져버리네 씨벌련아")

씨벌련아 0.08935724090607579
씨벌련이 0.0858298729359353
토씨하나 0.07181015790380983
조져버리네 0.04700403646018927
좆중복 0.030699808498733255
로 -0.005387206049654063
그대 0.004605442312728676
아침에 0.003470220259322148
안바꾸고 0.0031380933723337864
올린거 0.00228072761986878


'씨벌련이 닉변하고 아침에 올린거 토씨하나 안바꾸고  그대-로 좆중복 조져버리네 ****'